In [1]:
import sys
sys.path.append('./KPConv-PyTorch/')

In [2]:
from utils.config import Config
from datasets.CellData import CellDataset, CellDataCollate, CellDataCustomBatch, CellDataSampler
from models.architectures import KPFCNN
from torch.utils.data import DataLoader
from utils.trainer import ModelTrainer
import numpy as np
import torch

In [3]:
class CellDataConfig(Config):
    """
    Override the parameters you want to modify for this dataset
    """

    ####################
    # Dataset parameters
    ####################

    # Dataset name
    dataset = 'Fluo-C3DH-A549'

    # Number of classes in the dataset (This value is overwritten by dataset class when Initializating dataset).
    num_classes = None

    # Type of task performed on this dataset (also overwritten)
    dataset_task = 'cloud_segmentation'

    # Number of CPU threads for the input pipeline
    input_threads = 0

    #########################
    # Architecture definition
    #########################

    # Define layers
    architecture = ['simple',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary']

    ###################
    # KPConv parameters
    ###################

    # Radius of the input sphere
    in_radius = 2.5 #2.0

    # Number of kernel points
    num_kernel_points = 15

    # Size of the first subsampling grid in meter
    first_subsampling_dl = 0.08

    # Radius of convolution in "number grid cell". (2.5 is the standard value)
    conv_radius = 3.0 #2.5

    # Radius of deformable convolution in "number grid cell". Larger so that deformed kernel can spread out
    deform_radius = 6.0

    # Radius of the area of influence of each kernel point in "number grid cell". (1.0 is the standard value)
    KP_extent = 1.5

    # Behavior of convolutions in ('constant', 'linear', 'gaussian')
    KP_influence = 'gaussian' #linear

    # Aggregation function of KPConv in ('closest', 'sum')
    aggregation_mode = 'sum'

    # Choice of input features
    first_features_dim = 128
    in_features_dim = 1

    # Can the network learn modulations
    modulated = False

    # Batch normalization parameters
    use_batch_norm = True
    batch_norm_momentum = 0.02

    # Deformable offset loss
    # 'point2point' fitting geometry by penalizing distance from deform point to input points
    # 'point2plane' fitting geometry by penalizing distance from deform point to input point triplet (not implemented)
    deform_fitting_mode = 'point2point'
    deform_fitting_power = 1.0              # Multiplier for the fitting/repulsive loss
    deform_lr_factor = 0.1                  # Multiplier for learning rate applied to the deformations
    repulse_extent = 1.2                    # Distance of repulsion for deformed kernel points

    #####################
    # Training parameters
    #####################

    # Maximal number of epochs
    max_epoch = 500

    # Learning rate management
    learning_rate = 1e-5
    momentum = 0.98
    lr_decays = {i: 0.1 ** (1 / 150) for i in range(1, max_epoch)}
    grad_clip_norm = 100.0

    # Number of batch
    batch_num = 10

    # Number of steps per epochs
    epoch_steps = 100

    # Number of validation examples per epoch
    validation_size = 50

    # Number of epoch between each checkpoint
    checkpoint_gap = 10

    # Augmentations
    augment_scale_anisotropic = True
    augment_symmetries = [True, False, False]
    augment_rotation = 'vertical'
    augment_scale_min = 0.8
    augment_scale_max = 1.2
    augment_noise = 0.001
    augment_color = 0.8

    # The way we balance segmentation loss
    #   > 'none': Each point in the whole batch has the same contribution.
    #   > 'class': Each class has the same contribution (points are weighted according to class balance)
    #   > 'batch': Each cloud in the batch has the same contribution (points are weighted according cloud sizes)
    segloss_balance = 'class'
    class_w = [0.03, 0.97]

    # Do we need to save convergence
    saving = True
    saving_path = None

In [4]:
config = CellDataConfig()
train_dataset = CellDataset(config, set = 'training')
validation_dataset = CellDataset(config, set = 'validation')

training_sampler = CellDataSampler(train_dataset)
validation_sampler = CellDataSampler(validation_dataset)


Preparing KDTree for cloud t000, subsampled at 0.080
14.7 MB loaded in 3.2s

Preparing KDTree for cloud t002, subsampled at 0.080
14.7 MB loaded in 3.3s

Preparing KDTree for cloud t006, subsampled at 0.080
14.7 MB loaded in 3.3s

Preparing KDTree for cloud t008, subsampled at 0.080
14.7 MB loaded in 3.3s

Preparing KDTree for cloud t012, subsampled at 0.080
14.7 MB loaded in 3.3s

Preparing KDTree for cloud t013, subsampled at 0.080
14.7 MB loaded in 3.2s

Preparing KDTree for cloud t014, subsampled at 0.080
14.7 MB loaded in 3.2s

Preparing KDTree for cloud t019, subsampled at 0.080
14.7 MB loaded in 3.3s

Preparing KDTree for cloud t022, subsampled at 0.080
14.7 MB loaded in 3.3s

Preparing KDTree for cloud t025, subsampled at 0.080
14.7 MB loaded in 3.2s

Preparing KDTree for cloud t026, subsampled at 0.080
14.7 MB loaded in 3.1s

Preparing KDTree for cloud t027, subsampled at 0.080
14.7 MB loaded in 3.2s

Preparing KDTree for cloud t028, subsampled at 0.080
14.7 MB loaded in 2.8s

In [5]:
training_loader = DataLoader(train_dataset,
                            collate_fn=CellDataCollate,
                            sampler=training_sampler,)

validation_loader = DataLoader(validation_dataset,
                              collate_fn=CellDataCollate,
                              sampler=training_sampler,)

training_sampler.calibration(training_loader)
validation_sampler.calibration(validation_loader)

net = KPFCNN(config, train_dataset.label_values, train_dataset.ignored_labels)
trainer = ModelTrainer(net, config)


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s



In [ ]:
trainer.train(net, training_loader, validation_loader, config)

e000-i0000 => L=15.068 acc= 48% / t(ms):  33.9 1305.2 288.6)
e000-i0002 => L=14.836 acc= 32% / t(ms):  40.4 239.1 304.2)
e000-i0004 => L=15.051 acc= 42% / t(ms):  39.5 241.3 300.4)
e000-i0006 => L=15.017 acc= 44% / t(ms):  50.3 237.9 297.8)
e000-i0008 => L=15.172 acc= 37% / t(ms):  47.2 236.4 296.1)
e000-i0010 => L=14.814 acc= 46% / t(ms):  44.5 235.0 293.5)
e000-i0012 => L=14.990 acc= 35% / t(ms):  42.5 233.6 292.8)
e000-i0014 => L=14.763 acc= 44% / t(ms):  40.5 234.2 290.2)
e000-i0016 => L=14.783 acc= 49% / t(ms):  38.9 232.3 288.7)
e000-i0018 => L=15.328 acc= 40% / t(ms):  38.1 235.0 287.9)
e000-i0020 => L=14.944 acc= 44% / t(ms):  36.9 238.7 287.4)
e000-i0022 => L=14.879 acc= 47% / t(ms):  79.6 237.9 285.8)
e000-i0024 => L=14.532 acc= 46% / t(ms):  71.4 237.5 285.9)
e000-i0026 => L=14.878 acc= 45% / t(ms):  64.3 237.7 285.7)
e000-i0028 => L=14.831 acc= 44% / t(ms):  59.3 238.8 285.1)
e000-i0030 => L=14.723 acc= 40% / t(ms):  55.1 237.8 285.2)
e000-i0032 => L=14.422 acc= 46% / t(ms)

e002-i0024 => L=13.218 acc= 87% / t(ms):  41.3 298.8 338.1)
e002-i0026 => L=13.429 acc= 91% / t(ms):  39.6 290.8 330.5)
e002-i0028 => L=13.724 acc= 78% / t(ms):  38.3 283.2 325.3)
e002-i0030 => L=13.163 acc= 90% / t(ms):  37.3 276.7 319.0)
e002-i0032 => L=13.397 acc= 87% / t(ms):  36.8 272.3 314.3)
e002-i0034 => L=13.629 acc= 89% / t(ms): 129.8 267.9 310.9)
e002-i0036 => L=13.183 acc= 88% / t(ms): 112.7 263.3 307.4)
e002-i0038 => L=13.761 acc= 91% / t(ms):  98.2 260.9 310.4)
e002-i0040 => L=13.008 acc= 91% / t(ms):  86.8 258.9 307.2)
e002-i0042 => L=13.487 acc= 91% / t(ms):  77.5 260.8 304.4)
e002-i0044 => L=13.488 acc= 91% / t(ms):  69.1 261.7 305.8)
e002-i0046 => L=13.319 acc= 88% / t(ms):  62.2 255.4 304.3)
e002-i0048 => L=13.388 acc= 86% / t(ms):  56.6 253.2 302.3)
e002-i0050 => L=13.451 acc= 93% / t(ms):  52.0 252.4 300.5)
e002-i0052 => L=13.384 acc= 88% / t(ms):  49.4 253.5 300.6)
e002-i0054 => L=13.746 acc= 88% / t(ms):  46.0 258.3 306.1)
e002-i0056 => L=13.434 acc= 89% / t(ms):

e004-i0042 => L=12.893 acc= 93% / t(ms):  82.4 297.2 346.8)
e004-i0044 => L=12.982 acc= 98% / t(ms):  73.1 288.3 336.4)
e004-i0046 => L=12.562 acc= 89% / t(ms):  65.7 280.2 329.4)
e004-i0048 => L=13.034 acc= 97% / t(ms):  58.9 274.9 324.2)
e004-i0050 => L=12.768 acc= 98% / t(ms):  53.9 274.4 318.4)
e004-i0052 => L=12.421 acc= 91% / t(ms):  50.3 267.4 312.6)
e004-i0054 => L=13.108 acc= 88% / t(ms):  46.8 263.6 308.5)
e004-i0056 => L=12.866 acc= 92% / t(ms):  44.0 261.1 309.6)
e004-i0058 => L=12.521 acc=100% / t(ms):  41.6 259.5 304.8)
e004-i0060 => L=12.698 acc=100% / t(ms):  40.4 256.6 300.6)
e004-i0062 => L=12.542 acc= 96% / t(ms):  39.4 253.5 302.1)
e004-i0064 => L=12.315 acc= 95% / t(ms):  46.4 251.3 301.0)
e004-i0066 => L=12.876 acc=100% / t(ms):  44.7 249.4 301.2)
e004-i0068 => L=12.470 acc= 99% / t(ms):  43.3 253.0 301.6)
e004-i0070 => L=12.495 acc=100% / t(ms):  42.1 256.6 303.0)
e004-i0072 => L=12.718 acc= 99% / t(ms):  41.9 262.7 310.4)
e004-i0074 => L=12.829 acc= 95% / t(ms):

e006-i0063 => L=12.835 acc= 89% / t(ms):  45.5 271.7 321.4)
e006-i0065 => L=12.890 acc= 94% / t(ms):  45.3 285.6 326.6)
e006-i0067 => L=12.148 acc= 93% / t(ms):  45.4 288.4 331.3)
e006-i0069 => L=12.405 acc= 82% / t(ms):  51.5 288.5 341.0)
e006-i0071 => L=12.106 acc= 99% / t(ms):  77.9 288.2 341.4)
e006-i0073 => L=12.440 acc= 94% / t(ms):  70.8 291.1 343.5)
e006-i0075 => L=12.479 acc= 87% / t(ms):  66.0 295.8 345.5)
e006-i0077 => L=11.923 acc=100% / t(ms):  61.6 301.3 345.4)
e006-i0079 => L=12.315 acc=100% / t(ms):  57.7 301.6 347.0)
e006-i0081 => L=11.762 acc= 99% / t(ms):  55.3 299.7 350.0)
e006-i0083 => L=12.610 acc= 93% / t(ms):  52.5 299.5 350.6)
e006-i0085 => L=12.301 acc= 91% / t(ms):  51.4 301.9 350.7)
e006-i0087 => L=12.067 acc= 99% / t(ms):  50.2 301.4 353.1)
e006-i0089 => L=12.092 acc=100% / t(ms):  48.6 301.4 354.2)
e006-i0091 => L=11.984 acc=100% / t(ms):  46.5 297.2 344.3)
e006-i0093 => L=12.486 acc= 93% / t(ms):  43.9 290.4 335.3)
e006-i0095 => L=12.402 acc= 88% / t(ms):

e008-i0080 => L=11.550 acc=100% / t(ms):  37.6 276.6 313.7)
e008-i0082 => L=11.905 acc= 86% / t(ms):  39.4 283.2 321.1)
e008-i0084 => L=11.655 acc=100% / t(ms):  40.6 285.6 336.0)
e008-i0086 => L=11.474 acc=100% / t(ms):  41.5 293.1 338.6)
e008-i0088 => L=11.388 acc= 95% / t(ms):  42.0 295.4 342.3)
e008-i0090 => L=12.436 acc= 89% / t(ms):  42.0 300.8 349.0)
e008-i0092 => L=11.427 acc= 97% / t(ms):  41.7 301.5 360.5)
e008-i0094 => L=11.676 acc=100% / t(ms):  42.1 306.6 360.4)
e008-i0096 => L=12.232 acc= 86% / t(ms):  43.1 306.0 358.3)
e008-i0098 => L=11.770 acc= 98% / t(ms):  43.1 307.5 357.8)
Validation : 12.0% (timings : 12.23 34.42)
Validation : 24.0% (timings : 24.06 60.53)
Validation : 36.0% (timings : 32.57 78.83)
Validation : 48.0% (timings : 38.65 93.01)
Validation : 60.0% (timings : 42.40 103.98)
Validation : 72.0% (timings : 44.90 111.63)
Validation : 86.0% (timings : 44.29 111.53)
Validation : 100.0% (timings : 42.98 110.64)
Validation : 114.0% (timings : 42.20 109.59)
Valida

...

e036-i0058 => L=10.184 acc= 92% / t(ms):  41.0 287.0 328.7)
e036-i0060 => L=9.719 acc= 91% / t(ms):  39.5 280.4 321.2)
e036-i0062 => L=9.925 acc= 94% / t(ms):  38.0 274.8 316.6)
e036-i0064 => L=9.960 acc= 92% / t(ms):  36.9 271.1 311.3)
e036-i0066 => L=11.409 acc= 92% / t(ms):  36.6 268.1 308.1)
e036-i0068 => L=9.893 acc=100% / t(ms):  35.7 266.6 308.1)
e036-i0070 => L=9.869 acc=100% / t(ms):  35.9 269.3 314.5)
e036-i0072 => L=9.861 acc= 99% / t(ms):  37.2 275.9 321.2)
e036-i0074 => L=10.100 acc= 94% / t(ms):  39.7 283.1 326.0)
e036-i0076 => L=9.998 acc=100% / t(ms):  41.0 290.6 332.0)
e036-i0078 => L=11.053 acc= 81% / t(ms):  41.4 292.6 338.3)
e036-i0080 => L=9.916 acc= 99% / t(ms):  41.8 295.2 339.9)
e036-i0082 => L=9.802 acc= 92% / t(ms):  42.0 298.1 344.1)
e036-i0084 => L=9.829 acc= 98% / t(ms):  42.6 299.0 352.0)
e036-i0086 => L=10.408 acc= 87% / t(ms):  42.6 301.0 355.2)
e036-i0088 => L=11.059 acc= 82% / t(ms):  43.5 309.0 361.8)
e036-i0090 => L=10.853 acc= 83% / t(ms):  44.3 313

e038-i0078 => L=9.841 acc=100% / t(ms):  44.0 310.5 352.9)
e038-i0080 => L=9.505 acc=100% / t(ms):  43.4 309.6 354.6)
e038-i0082 => L=9.734 acc= 88% / t(ms):  43.4 309.3 354.3)
e038-i0084 => L=9.600 acc= 89% / t(ms):  43.1 308.7 355.6)
e038-i0086 => L=9.778 acc=100% / t(ms):  42.8 308.4 357.7)
e038-i0088 => L=9.770 acc= 89% / t(ms):  45.7 318.9 367.2)
e038-i0090 => L=9.760 acc= 98% / t(ms):  47.2 322.9 373.8)
e038-i0092 => L=9.873 acc= 96% / t(ms):  49.1 328.9 382.0)
e038-i0094 => L=10.372 acc= 88% / t(ms):  50.7 331.6 389.6)
e038-i0096 => L=9.789 acc= 98% / t(ms):  51.6 334.9 389.5)
e038-i0098 => L=9.836 acc=100% / t(ms):  52.0 330.5 387.8)
Validation : 12.0% (timings : 12.15 33.66)
Validation : 26.0% (timings : 21.71 59.01)
Validation : 40.0% (timings : 28.64 75.52)
Validation : 54.0% (timings : 33.17 88.67)
Validation : 68.0% (timings : 35.66 97.93)
Validation : 80.0% (timings : 39.74 104.70)
Validation : 92.0% (timings : 43.92 113.13)
Validation : 104.0% (timings : 47.98 119.56)
Va

e040-i0096 => L=9.813 acc= 83% / t(ms):  36.0 265.2 310.6)
e040-i0098 => L=9.648 acc= 90% / t(ms):  36.5 269.0 312.9)
Validation : 10.0% (timings : 14.31 36.09)
Validation : 22.0% (timings : 25.99 62.63)
Validation : 34.0% (timings : 34.32 82.36)
Validation : 46.0% (timings : 40.01 97.31)
Validation : 58.0% (timings : 43.90 106.05)
Validation : 70.0% (timings : 47.05 112.76)
Validation : 82.0% (timings : 50.02 118.13)
Validation : 94.0% (timings : 52.01 121.37)
Validation : 106.0% (timings : 53.48 125.20)
Validation : 118.0% (timings : 54.00 127.02)
Validation : 130.0% (timings : 54.22 128.27)
Validation : 142.0% (timings : 54.67 129.81)
Validation : 154.0% (timings : 54.31 130.33)
Validation : 166.0% (timings : 54.39 130.51)
Validation : 178.0% (timings : 55.22 130.89)
Validation : 190.0% (timings : 55.08 131.00)
Confusion Matrix
****************
[[471350.06    40654.973 ]
 [  2537.8733   9745.126 ]]
[0.91605586 0.18408574]
Fluo-C3DH-A549 mean IoU = 55.0%
e041-i0000 => L=9.989 acc= 91

Validation : 158.0% (timings : 45.18 118.85)
Validation : 170.0% (timings : 48.60 122.34)
Validation : 182.0% (timings : 50.68 127.87)
Validation : 194.0% (timings : 52.11 129.42)
Confusion Matrix
****************
[[466161.72    45843.25  ]
 [  1865.4877  10417.513 ]]
[0.907158   0.17922215]
Fluo-C3DH-A549 mean IoU = 54.3%
e043-i0000 => L=10.910 acc= 81% / t(ms): 17222.7 358.4 351.1)
e043-i0002 => L=9.937 acc= 99% / t(ms):  45.2 342.7 351.3)
e043-i0004 => L=9.774 acc= 89% / t(ms):  44.8 334.3 352.6)
e043-i0006 => L=9.781 acc= 94% / t(ms):  44.5 328.6 350.3)
e043-i0008 => L=9.703 acc=100% / t(ms):  44.6 324.7 350.1)
e043-i0010 => L=9.812 acc= 92% / t(ms):  44.3 320.8 350.3)
e043-i0012 => L=9.753 acc= 90% / t(ms):  43.7 319.4 349.9)
e043-i0014 => L=9.583 acc=100% / t(ms):  43.8 318.2 349.5)
e043-i0016 => L=10.653 acc= 82% / t(ms):  44.4 317.2 350.7)
e043-i0018 => L=9.852 acc= 88% / t(ms):  44.2 314.0 348.9)
e043-i0020 => L=9.877 acc= 89% / t(ms):  44.1 310.4 348.7)
e043-i0022 => L=10.376

e045-i0008 => L=10.050 acc= 90% / t(ms):  37.5 260.4 299.6)
e045-i0010 => L=9.563 acc= 99% / t(ms):  38.3 268.2 302.6)
e045-i0012 => L=9.497 acc= 97% / t(ms):  38.9 268.7 305.8)
e045-i0014 => L=9.746 acc=100% / t(ms):  39.7 275.5 312.1)
e045-i0016 => L=9.825 acc= 90% / t(ms):  40.4 281.2 318.4)
